# Final Version with improvements



In [174]:
# CMPT459 2020 spring
# milestone2
# data cleaned and feature selection and numericalization for classifier
# group name: Salt

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
%matplotlib inline
from sklearn import model_selection, preprocessing, ensemble
from sklearn.metrics import log_loss
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import csv as csv
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multiclass import OneVsOneClassifier
from sklearn.model_selection import train_test_split

def outlier_fd(data, threshold=0.5):
    low, high = np.percentile(data,1), np.percentile(data, 99)
    dist = high - low
    cut_off = dist * threshold
    lower, upper = low-cut_off, high+cut_off
    return [lower, upper]

def photos_number_counter(x):
    return len(x)

def valid_des(s):
    valid_s = " "
    for i in s.split():
        if i.isalnum():
            valid_s = " ".join([valid_s, i])
            
    return valid_s.strip()



In [162]:
train = pd.read_json('train.json')
test = pd.read_json('test.json')
ptrain = train
ptest = test
ptrain = ptrain.drop(['interest_level'], axis = 1)
pwhole = ptrain.append(ptest)

# Data Prepocessing for training data and test data

In [163]:
# Find the outlier of price, latitude and longitude using bound given by outlier_fd
[price_lower, price_higher] = outlier_fd(ptrain['price'])
[latitude_lower, latitude_higher] = outlier_fd(ptrain['latitude'],1)
[longitude_lower, longitude_higher] = outlier_fd(ptrain['longitude'],1)
#deal with outliers, set the column for modified data and plot again
pwhole['price_modify'] = pwhole['price']
pwhole['price_modify'].loc[pwhole['price_modify']<price_lower] = price_lower
pwhole['price_modify'].loc[pwhole['price_modify']>price_higher] = price_higher
#deal with outliers, set the column for modified data and plot again
pwhole['latitude_modify'] = pwhole['latitude']
pwhole['latitude_modify'].loc[pwhole['latitude_modify']<latitude_lower] = latitude_lower
pwhole['latitude_modify'].loc[pwhole['latitude_modify']>latitude_higher] = latitude_higher
#deal with outliers, set the column for modified data and plot again
pwhole['longitude_modify'] = pwhole['longitude']
pwhole['longitude_modify'].loc[pwhole['longitude']<longitude_lower] = longitude_lower
pwhole['longitude_modify'].loc[pwhole['longitude']>longitude_higher] = longitude_higher
#get the number of photos
pwhole['photos_number'] = pwhole['photos'].apply(photos_number_counter)
#set all feature words in tf_idf as features for train for feature_modify
pwhole['features_modify'] = pwhole['features'].apply(lambda x:" ".join(["_".join(i.split(" "))for i in x]))
counterf = CountVectorizer(max_features=100)
train_sparse_f = counterf.fit_transform(pwhole['features_modify'])
feature_list = counterf.get_feature_names()
array1 = train_sparse_f.toarray()
for i, feature_str in enumerate(feature_list):
    list_all = []
    [rows, cols] = array1.shape
    for row in range(rows):
        list_all.append(array1[row][i])
    feature_str_ = feature_str + '(f)'
    feature_v = pd.Series(list_all,pwhole.index, name = feature_str_)
    pwhole[feature_str_] = feature_v
#set all feature words in tf_idf as features for train for description_modify
pwhole['description_modify'] = pwhole['description'].apply(lambda x:valid_des(x))
counterd = CountVectorizer(stop_words=['features'], max_features=100)
train_sparse_d = counterd.fit_transform(pwhole['description_modify'])
description_list = counterd.get_feature_names()
array2 = train_sparse_d.toarray()
for i, description_str in enumerate(description_list):
    list_all = []
    [rows, cols] = array2.shape
    for row in range(rows):
        list_all.append(array2[row][i])
    description_str_ = description_str + '(d)'
    description_v = pd.Series(list_all,pwhole.index, name = description_str_)
    pwhole[description_str_] = description_v
## /*add something to change the created to year month day*/
pwhole['created']=pd.to_datetime(pwhole["created"])
pwhole['year']=pwhole['created'].dt.year
pwhole['month']=pwhole['created'].dt.month
pwhole['day']=pwhole['created'].dt.day



/Users/lewislin/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


# Feature Selection

In [164]:
pwhole = pwhole.drop(['central_a(f)','in_super(f)','on(f)','pre(f)','war(f)'], axis = 1)
pwhole = pwhole.drop(['an(d)','and(d)','are(d)','as(d)','at(d)','be(d)','by(d)','can(d)',
                      'for(d)','from(d)','in(d)','is(d)','it(d)','just(d)','me(d)','more(d)',
                      'natural(d)','new(d)','or(d)','park(d)','that(d)','the(d)','this(d)',
                      'to(d)','will(d)','with(d)','you(d)','your(d)'], axis = 1)
pwhole = pwhole.drop(["building_id", "created", "description", "display_address", "features", 
                     "manager_id", "photos", "street_address", "price", "longitude",
                        "latitude", "features_modify", "description_modify"], axis=1)


In [165]:
X = pwhole[:49352]
X_Test = pwhole[49352:]
X = X.drop(['listing_id'], axis = 1)
X_Test = X_Test.drop(['listing_id'], axis = 1)
Y = pd.get_dummies(train['interest_level'])


Now we have three datasets:

training samples(without labels) -- X

training labels -- Y

test dataset -- X_Test


the columns of X_train is shown below:

In [166]:
X_train

,bathrooms,bedrooms,price_modify,latitude_modify,longitude_modify,photos_number,_photos(f),actual_apt(f),balcony(f),bike_room(f),...,supports(d),text(d),two(d),unit(d),west(d),windows(d),york(d),year,month,day
id,,,,,,,,,,,,,,,,,,,,,
0,1.0,1,2400.0,40.7108,-73.9539,12,0,0,0,0,...,0,1,0,0,0,0,0,2016,6,16
1,1.0,2,3800.0,40.7513,-73.9722,6,0,0,0,0,...,0,1,0,0,0,0,0,2016,6,1
2,1.0,2,3495.0,40.7575,-73.9625,6,0,0,0,0,...,0,0,0,0,0,0,0,2016,6,14
3,1.5,3,3000.0,40.7145,-73.9425,5,0,0,0,0,...,0,0,0,0,0,0,0,2016,6,24
4,1.0,0,2795.0,40.7439,-73.9743,4,0,0,0,0,...,0,0,0,0,0,0,0,2016,6,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49347,1.0,3,2800.0,40.8433,-73.9396,5,0,0,0,0,...,1,0,0,0,0,0,1,2016,4,5
49348,1.0,2,2395.0,40.8198,-73.9578,5,0,0,0,0,...,0,0,0,0,0,0,1,2016,4,2
49349,1.0,1,1850.0,40.5765,-73.9554,3,0,0,0,0,...,0,0,0,0,0,0,0,2016,4,26


# 5-fold Cross-validation (repeats 5 times)

I use 5-fold cross-validation here and choose the classifer with lowest log loss(Best_OVR1), highest classification accuracy(Best_OVR2), highest precision(Best_OVR3)

In [167]:
training_score = 0
validation_score = 0
Best_validation_score = 20
count = 0
total_validation_score = 0
precision = []
recall = []
test_accuracy =[]
test_accuracy_value = 0
test_precision = 0

from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import log_loss
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import precision_score, recall_score
# from sklearn.model_selection import cross_val_score
# scrores
#maybe 5-fold is better than 10-fold
kf = RepeatedKFold(n_splits=5, n_repeats=5, random_state=None) 

for train_index, test_index in kf.split(X):
#     print("Train:", train_index, "Validation:",test_index)
    X['id']=list(range(0,49352))
    X.set_index("id", inplace=True)
    Y['id']=list(range(0,49352))
    Y.set_index("id", inplace=True)
    X_train, X_test = X.loc[train_index,:], X.loc[test_index,:]
    Y_train, Y_test = Y.loc[train_index,:], Y.loc[test_index,:]
    OVR = OneVsRestClassifier(LogisticRegression(warm_start=True,max_iter=200,tol=0.00001)).fit(X_train,Y_train)
#   classification accuracy metrics
    if test_accuracy_value<OVR.score(X_test,Y_test):
        test_accuracy_value=OVR.score(X_test,Y_test)
        Best_OVR2 = OVR
    test_accuracy.append(OVR.score(X_test,Y_test))
#   log loss on training dataset
    Y_Prediction=OVR.predict_proba(X_train)
    Y_Prediction=pd.DataFrame(Y_Prediction)
    Y_train=pd.DataFrame(Y_train)
    training_score=log_loss(Y_train, Y_Prediction)
    print("training log_loss: %0.3f" % training_score)

#   log loss on test dataset
    Y_Prediction=OVR.predict_proba(X_test)
    Y_Prediction=pd.DataFrame(Y_Prediction)
    Y_test=pd.DataFrame(Y_test)
    validation_score=log_loss(Y_test, Y_Prediction)
    count+=1
    total_validation_score+=validation_score
    print("validation log_loss: %0.3f" % validation_score)

#   logloss metrics
    if validation_score<Best_validation_score:
        Best_validation_score = validation_score
        Best_OVR1 = OVR

    if count==5:
        average = total_validation_score/float(count)
        print("average validation log_loss: %0.3f" % average)
        count=0
        total_validation_score=0
    
    Y_Prediction=OVR.predict(X_test)
#   print confusion matrix
    print(multilabel_confusion_matrix(Y_test, Y_Prediction))
#   confusion matrix matrics
    if test_precision<precision_score(Y_test, Y_Prediction, average='macro'):
        test_precision=precision_score(Y_test, Y_Prediction, average='macro')
        Best_OVR3 = OVR
    precision.append(precision_score(Y_test, Y_Prediction, average='macro'))
    recall.append(recall_score(Y_test, Y_Prediction, average='macro'))
print ("One vs rest best validation accuracy: %0.3f" % Best_validation_score) 
print(precision)
print(recall)
print (test_accuracy)

/Users/lewislin/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/lewislin/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/lewislin/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


training log_loss: 0.669
validation log_loss: 0.676
[[[9072   14]
  [ 762   23]]

 [[1126 1895]
  [ 682 6168]]

 [[7432  203]
  [2078  158]]]


/Users/lewislin/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/lewislin/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/lewislin/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


training log_loss: 0.673
validation log_loss: 0.662
[[[9102   22]
  [ 731   16]]

 [[1096 1846]
  [ 670 6259]]

 [[7515  161]
  [2066  129]]]


/Users/lewislin/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/lewislin/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/lewislin/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


...

training log_loss: 0.673
validation log_loss: 0.659
[[[9075   26]
  [ 749   20]]

 [[1081 1857]
  [ 640 6292]]

 [[7540  161]
  [2036  133]]]


/Users/lewislin/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/lewislin/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/lewislin/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


training log_loss: 0.666
validation log_loss: 0.685
[[[9078   27]
  [ 741   24]]

 [[1082 1921]
  [ 743 6124]]

 [[7431  201]
  [2091  147]]]


/Users/lewislin/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/lewislin/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/lewislin/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


training log_loss: 0.667
validation log_loss: 0.682
average validation log_loss: 0.676
[[[9063   30]
  [ 741   36]]

 [[1140 1949]
  [ 708 6073]]

 [[7347  211]
  [2151  161]]]
One vs rest best validation accuracy: 0.659
[0.6080901890896109, 0.546039858735524, 0.5741532467770729, 0.5269728263704168, 0.5250584943126073, 0.5762669994658779, 0.5516340836719027, 0.5246135256358105, 0.5484393887365052, 0.5825944178633587, 0.519902405323029, 0.5919979081918355, 0.557953820851782, 0.5354165064207398, 0.532915243929496, 0.5607819489839135, 0.6018425844321852, 0.5175137503486993, 0.5994253987359415, 0.5312578834925817, 0.5498866715415179, 0.5332613783674398, 0.5530942798385589, 0.5514067254385092, 0.578431125255986]
[0.33346640516833204, 0.3278312970809841, 0.33406113553268973, 0.3280493912008187, 0.33391707613568333, 0.33490136317768165, 0.3280035924542935, 0.32989467611537765, 0.3322798213858156, 0.333778473604277, 0.331862691229355, 0.33404429273994496, 0.33249379685733865, 0.327560746471601

# Write into submission.csv

Here I use the best classifier evaluated by log loss

In [168]:
Y_Prediction=Best_OVR1.predict_proba(X_Test)  #use the best classifier evaluated by log loss
y=pd.DataFrame(Y_Prediction)
y.reset_index(drop=True, inplace=True)
ids = test['listing_id']
ids.reset_index(drop=True, inplace=True)
out = pd.concat([ids, y],axis=1, ignore_index=True)
out
# out.columns=['listing_id','high','medium','low']
out.columns=['listing_id','high','low','medium']
submission_file = open("submission.csv", "w")
out.to_csv('submission.csv', index = False)
submission_file.close()

# First Version

Here is mainly first version and I did some feature selections corresponding to the modification part(a) in the report.

In [169]:
def dataforclf(dataset):
    train = pd.read_json(dataset)
    #Find the outlier of price, latitude and longitude using bound given by outlier_fd
    [price_lower, price_higher] = outlier_fd(train['price'])
    outlier_price = train[(train['price']<price_lower)|(train['price']>price_higher)]
    [latitude_lower, latitude_higher] = outlier_fd(train['latitude'],1)
    outlier_latitude = train[(train['latitude']<latitude_lower)|(train['latitude']>latitude_higher)]
    [longitude_lower, longitude_higher] = outlier_fd(train['longitude'],1)
    outlier_longitude = train[(train['longitude']<longitude_lower)|(train['longitude']>longitude_higher)]
    #deal with outliers, set the column for modified data and plot again
    train['price_modify'] = train['price']
    train['price_modify'].loc[train['price_modify']<price_lower] = price_lower
    train['price_modify'].loc[train['price_modify']>price_higher] = price_higher
    #deal with outliers, set the column for modified data and plot again
    train['latitude_modify'] = train['latitude']
    train['latitude_modify'].loc[train['latitude_modify']<latitude_lower] = latitude_lower
    train['latitude_modify'].loc[train['latitude_modify']>latitude_higher] = latitude_higher
    #deal with outliers, set the column for modified data and plot again
    train['longitude_modify'] = train['longitude']
    train['longitude_modify'].loc[train['longitude']<longitude_lower] = longitude_lower
    train['longitude_modify'].loc[train['longitude']>longitude_higher] = longitude_higher
    #get the number of photos
    train['photos_number'] = train['photos'].apply(photos_number_counter)
    #set all feature words in tf_idf as features for train for feature_modify
    train['features_modify'] = train['features'].apply(lambda x:" ".join(["_".join(i.split(" "))for i in x]))
    tfidf_f = TfidfVectorizer(stop_words=['central_a', 'pre', 'war','features'], max_features=98)
    train_sparse_f = tfidf_f.fit_transform(train['features_modify'])
    feature_list = tfidf_f.get_feature_names()
    array1 = train_sparse_f.toarray()
    for i, feature_str in enumerate(feature_list):
        list_all = []
        [rows, cols] = array1.shape
        for row in range(rows):
            list_all.append(array1[row][i])
        feature_v = pd.Series(list_all,train.index, name = feature_str)
        train[feature_str] = feature_v
    #set all feature words in tf_idf as features for train for description_modify
    train['description_modify'] = train['description'].apply(lambda x:valid_des(x))
    tfidf_d = TfidfVectorizer(stop_words=['features'], max_features=100)
    train_sparse_d = tfidf_d.fit_transform(train['description_modify'])
    description_list = tfidf_d.get_feature_names()
    array2 = train_sparse_d.toarray()
    for i, description_str in enumerate(description_list):
        list_all = []
        [rows, cols] = array2.shape
        for row in range(rows):
            list_all.append(array2[row][i])
        description_v = pd.Series(list_all,train.index, name = description_str)
        train[description_str] = description_v
    ## /*add something to change the created to year month day*/
    train['created']=pd.to_datetime(train["created"])
    #extract pieces
    train['year']=train['created'].dt.year
    train['month']=train['created'].dt.month
    train['day']=train['created'].dt.day
    ## /*add something to do feature selection*/
    train = train.drop(["building_id", "created", "description", "display_address", "features", 
                     "manager_id", "photos", "street_address", "price", "longitude",
                        "latitude", "features_modify", "description_modify",
                        "_photos", "exposed_brick", "actual_apt", "full", 
                        "in_superintendent", "in_super", "live_in_super","on","newly_renovated"
                       ], axis=1)

    return train

In [170]:
#main function
train = dataforclf('train.json')
test  = dataforclf('test.json')

/Users/lewislin/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/Users/lewislin/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [171]:
import csv as csv
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multiclass import OneVsOneClassifier
from sklearn.model_selection import train_test_split
X = train.drop(["listing_id","interest_level"],axis=1)
Y = pd.get_dummies(train['interest_level'])

In [172]:
training_score = 0
validation_score = 0
Best_validation_score = 20

from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import log_loss
# from sklearn.model_selection import cross_val_score
# scrores
#maybe 5-fold is better than 10-fold
kf = RepeatedKFold(n_splits=5, n_repeats=5, random_state=None) 
count = 0
total_validation_score = 0
for train_index, test_index in kf.split(X):
    X['id']=list(range(0,49352))
    X.set_index("id", inplace=True)
    Y['id']=list(range(0,49352))
    Y.set_index("id", inplace=True)
    X_train, X_test = X.loc[train_index,:], X.loc[test_index,:]
    Y_train, Y_test = Y.loc[train_index,:], Y.loc[test_index,:]
    OVR = OneVsRestClassifier(LogisticRegression()).fit(X_train,Y_train)

    Y_Prediction=OVR.predict_proba(X_train)
    Y_Prediction=pd.DataFrame(Y_Prediction)
    Y_train=pd.DataFrame(Y_train)
    training_score=log_loss(Y_train, Y_Prediction)
    print("training log_loss: %0.3f" % training_score)
    Y_Prediction=OVR.predict_proba(X_test)
    Y_Prediction=pd.DataFrame(Y_Prediction)
    Y_test=pd.DataFrame(Y_test)
    validation_score=log_loss(Y_test, Y_Prediction)
    count+=1
    total_validation_score+=validation_score
    print("validation log_loss: %0.3f" % validation_score)
    
    if validation_score<Best_validation_score:
        Best_validation_score = validation_score
        Best_OVR1 = OVR
#     print ("One vs rest validation accuracy: %0.3f" % validation_score)
    if count==5:
        average = total_validation_score/float(count)
        print("average validation log_loss: %0.3f" % average)
        count=0
        total_validation_score=0
print ("One vs rest best validation accuracy: %0.3f" % Best_validation_score)    




/Users/lewislin/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/lewislin/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/lewislin/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


training log_loss: 0.703
validation log_loss: 0.690


/Users/lewislin/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/lewislin/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/lewislin/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


training log_loss: 0.700
validation log_loss: 0.700


/Users/lewislin/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/lewislin/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/lewislin/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


...

training log_loss: 0.696
validation log_loss: 0.704


/Users/lewislin/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/lewislin/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/lewislin/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


training log_loss: 0.700
validation log_loss: 0.696


/Users/lewislin/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/lewislin/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/lewislin/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


training log_loss: 0.701
validation log_loss: 0.705
average validation log_loss: 0.704
One vs rest best validation accuracy: 0.690


In [173]:
X_Test = test.drop("listing_id", axis=1)
Y_Prediction=Best_OVR1.predict_proba(X_Test)
y=pd.DataFrame(Y_Prediction)
y.reset_index(drop=True, inplace=True)
ids = test['listing_id']
ids.reset_index(drop=True, inplace=True)
out = pd.concat([ids, y],axis=1, ignore_index=True)
out
# out.columns=['listing_id','high','medium','low']
out.columns=['listing_id','high','low','medium']
submission_file = open("submission.csv", "w")
out.to_csv('submission.csv', index = False)
submission_file.close()